In [1]:
import pandas as pd
from geopy import distance
import numpy as np

In [4]:
apt_geocode = pd.read_csv("C:\\Users\\Young Jin Park\\Documents\\GitHub\\RGTM\\Cleaned//merged_manhattan_geocode.csv")

In [11]:
apt_geocode["GeoCode"] = list(zip(apt_geocode.latitude, apt_geocode.longitude))

In [12]:
apt_geocode

,Address,latitude,longitude,GeoCode
0,"526 EAST 5TH STREET, New York, NY",40.724206,-73.983263,"(40.7242057, -73.983263)"
1,"264 EAST 7TH STREET, New York, NY",40.723280,-73.977568,"(40.7232798, -73.9775683)"
2,"243 EAST 7TH STREET, New York, NY",40.723939,-73.978200,"(40.7239392, -73.9782)"
3,"238 EAST 4TH STREET, New York, NY",40.723272,-73.983026,"(40.7232718, -73.9830259)"
4,"311 EAST 3RD STREET, New York, NY",40.721224,-73.978940,"(40.7212244, -73.9789396)"
5,"327 EAST 3RD STREET, New York, NY",40.720873,-73.978398,"(40.7208731, -73.9783978)"
6,"742 EAST 6TH STREET, New York, NY",40.722619,-73.977816,"(40.7226187, -73.9778157)"
7,"254 7TH STREET, New York, NY",40.723410,-73.977873,"(40.7234098, -73.9778733)"
8,"254 EAST 7TH STREET, New York, NY",40.723410,-73.977873,"(40.7234098, -73.9778733)"
9,"252 EAST 7TH STREET, New York, NY",40.723533,-73.977950,"(40.7235325, -73.9779504)"


In [ ]:
def calculate_price_in_x_miles(x):
    apt.geocode
    